In [271]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from matplotlib import pyplot
from newspaper import Article

%matplotlib inline

from collections import defaultdict
from datetime import timedelta, date, datetime
import json
import numpy as np
import math

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"


In [56]:
class ArticleQueryBuilder():
    def __init__(self, article):
        self.article = article

    def baseline(self):
        return Q("match", title=self.article['title']) & Q("match", body=self.article['title'])
    
    def simple_and_query(self):
        return Q("match", title=self.article['title']) & Q("match", body=self.article['body'])
        
        
    def simple_or_query(self):
        return Q("match", title=self.article['title']) | Q("match", body=self.article['body'])
    
    
    def date_filter(self, before=timedelta(weeks=2),
                            after=timedelta(weeks=8)):
        date = datetime.strptime(self.article['date'], '%Y-%m-%d')
        return Q("bool", filter=[Q('range', 
                            published={
                                'gte': date - before,
                                'lte': date + after
                            })])
    
    def location_query(self):
        return Q("bool", should=[
            Q("match", body=self.article['location']), 
            Q("match", title=self.article['location']),
            Q("match", officeName=self.article['location'])])

In [141]:
#Filtert die ersten k verschiedenen Suchergebnisse heraus, falls soviele vorhanden sind
def first_k_results(res, k):
    resF=[]
    anz=0
    for value in res:
        ignore=0
        if value.ID in resF:
            ignore=1
        if ignore ==0:
            resF.append(value)
            anz =anz +1
        if anz==k :
            return resF
    return resF

In [58]:
def max_score(prids,k):
    Relevanz=[]
    score=0
    length=len(prids)
    for value in prids:
        Relevanz.append(int(value['score']))
    Relevanz.sort(reverse=True)
    for i in range(k):
        if i<length:
            score=score+(2**Relevanz[i]-1)/(math.log(2+i,2))
        else:
            return score
    return score

In [59]:
def score(res, prids, k):
    score=0
    length=len(res)
    for i in range(length):
        gefunden=False
        if i>=k:
            return score
        aktID=res[i]['ID']
        for value in prids:
            if value['id']==aktID:
                gefunden=True
                score=score+(2**int(value['score'])-1)/(math.log(2+i,2))
        if gefunden==False:
            print("Das Eregnis mit Nummer "+aktID+" wurde nicht gefunden")
    return score
    
    

In [60]:
def mittel(vektor):
    laenge=len(vektor)
    wert=0
    for i in range(laenge):
        wert=wert+vektor[i]/laenge
    return wert
    

In [162]:
def prec_at_k(query_result, ids, k):
    anzahl=0
    nummerErgebnis=0
    for element in query_result:
        #print("")
        #print(element['ID'])
        for valuedID in ids:
            #print(valuedID['id'])
            #print(valuedID['score'])
            if valuedID['id']==element['ID'] and int(valuedID['score'])==3:
                anzahl=anzahl+1
        nummerErgebnis=nummerErgebnis+1
        if nummerErgebnis==k:
            return anzahl/k
    return anzahl/k
            


In [165]:
def hit_upto_k(query_result, ids, k):
    anzahl=0
    nummerErgebnis=0
    for element in query_result:
        #print("")
        #print(element['ID'])
        for valuedID in ids:
            #print(valuedID['id'])
            #print(valuedID['score'])
            if valuedID['id']==element['ID'] and int(valuedID['score'])==3:
                anzahl=1
        nummerErgebnis=nummerErgebnis+1
        if nummerErgebnis==k:
            return anzahl
    return anzahl
            


In [277]:
data_file_names = [ 'sport-fussball',
              'immigration',
                   'straftaten-anschlaege',
                   'unfaelle',
                   'demonstrationen',
                   'grossveranstaltungen'
                  ]

In [278]:
data = {}
for name in data_file_names:
    with open(name+".json") as data_file:
        topic_data = json.load(data_file)
        data[name] = topic_data

In [281]:
client = Elasticsearch(hosts='https://police-pr-data.elastic.robin-in.space', http_auth=('elastic', 'failing-species-latter-diffused-carried-lining'), timeout=30, use_ssl=True, port='')

In [114]:
#Beispiel wie die Suche funktioniert
article= data['immigration']['data'][0]['articles'][0]
ab= ArticleQueryBuilder(article)
query = ab.simple_and_query() & ab.location_query() & ab.date_filter()
s = Search(using=client)
res = s.query(query).execute()

In [273]:
#Für die Erstelleung des NDCG werden für jeden Nachrichtenartikel für zwei verschiedene Anfragen die obersten k Treffer in zufälliger Reihenfolge angezeigt, per Hand bewertet und anschließend das Verfahren per Hand bewertet
topic="grossveranstaltungen"
len(data[topic]['data'])
ereignisNummer=2
len(data[topic]['data'][ereignisNummer]['articles'])
artikelNummer=0
data[topic]['data'][ereignisNummer]['articles'][artikelNummer]['url']
bereitsBew=[]
for valuedID in data[topic]['data'][ereignisNummer]['pr-ids']:
    bereitsBew.append(valuedID['id'])
#bereitsBew



3

2

'https://www.wochenspiegellive.de/hunsruecknahe/bad-kreuznach/artikel/jahrmarkt-2015-polizei-und-drk-ziehen-bilanz-32916/'

In [274]:
#die ersten k Treffer der Suchanfragen über alle Artikel werden in zufälliger Reihenfolge angezeigt um sie zu bewerten
k=5
valueArticle=[]
for article in data[topic]['data'][ereignisNummer]['articles']:
    ab= ArticleQueryBuilder(article)
    queryBase=ab.baseline()
    queryImprove= ab.simple_and_query() & ab.location_query() & ab.date_filter()
    s = Search(using=client)
    res1=s.query(queryBase).execute()
    res2=s.query(queryImprove).execute()
    res1=first_k_results(res1,k)
    res2=first_k_results(res2,k)
    res=res1+res2
    for value in res:
        ignore=0
        aktId=value['ID']
        for element in valueArticle:
            if element['ID']==value['ID']:
                ignore=1
        for element in bereitsBew:
            if element==value['ID']:
                ignore=1
        if ignore==0:
            valueArticle.append(value)
        
length=len(valueArticle)
permutation=np.random.permutation(length)
#permutation
for i in range(length):
    j=permutation[i]
    print(valueArticle[j].ID)
    print(valueArticle[j].title)
    print(valueArticle[j].body)
    print()
    print("____________________________________________________")
    print("----------------------------------------------------")
    print()
        
        
    



672154
POL-SE: Schlägerei während des Jahrmarktes
Bad Segeberg (ots) - Am Freitag konnte die Polizei eine schwere
Massenschlägerei verhindern. Auslöser war offensichtlich eine
Körperverletzung. An weiteren körperlichen Auseinandersetzungen
nahmen etwa 50 Personen teil. Ein nachvollziehbares Motiv für die
aggressive Stimmung der Beteiligten war nicht erkennbar. Die
Beteiligten waren hochgradig alkoholisiert. Ein 17-jähriger
Jugendlicher aus Ulzburg wurde im Gesicht verletzt. Drei Männer
mussten in polizeiliches Gewahrsam verbracht werden.

   Am Freitag hielt sich der 17-jährige Jugendliche gemeinsam mit
seinen beiden gleichaltrigen Freunden auf dem Jahrmarkt in Henstedt-
mzburg auf. Eine unbedeutende Körperberührung mit einem Mann führte
zu einem relativ harmlosen Streitgespräch. Ein anderer zuvor
unbeteiligter 21-jähriger Mann aus Ulzburg mischte sich in die
Angelegenheit ohne erkennbaren Grund ein. Daraufhin trat ein weiterer
Mann von vorn an den Jugendlichen heran, beleidigte ihn un

In [286]:
# der Score wird für eine ganze Thematik ausgerechnet
k=1
anzArtikel=0
anzEreignisse=0
baseline=[]
improved=[]
baselinePrec=[]
improvedPrec=[]
baselineHit=[]
improvedHit=[]
data_files=[ 'sport-fussball',
              'immigration', 'straftaten-anschlaege', 'unfaelle', 'demonstrationen', 'grossveranstaltungen'
                  ]
for topic in data_files:
    for ereignis in data[topic]['data']:
        anzEreignisse=anzEreignisse+1
        for article in ereignis['articles']:
            anzArtikel=anzArtikel+1
            ab= ArticleQueryBuilder(article)
            queryBase=ab.baseline()
            queryImprove= ab.simple_and_query() & ab.location_query() & ab.date_filter()
            s = Search(using=client)
            res1=s.query(queryBase).execute()
            #for value in res1:
            #    print(value['ID'])
            res2=s.query(queryImprove).execute()
            #for value in res2:
                #print(value['ID'])
            valuedIDs=ereignis['pr-ids']
            maxScore=max_score(valuedIDs, k)
            print("der maximale Score ist " +str(maxScore))
            res1=first_k_results(res1,k)
            #for value in res1:
            #    print(value['ID'])
            res2=first_k_results(res2,k)
            #res1[1]['ID']
            score1=score(res1, valuedIDs,k)
            print("baseline score ist " +str(score1))
            baseline.append(score1/maxScore)
            scorePrec1=prec_at_k(res1, valuedIDs, k)
            baselinePrec.append(scorePrec1)
            scoreHit1=hit_upto_k(res1, valuedIDs, k)
            baselineHit.append(scoreHit1)
            score2=score(res2, valuedIDs,k)
            print("verbesserter score ist " +str(score2))
            improved.append(score2/maxScore)
            scorePrec2=prec_at_k(res2, valuedIDs, k)
            improvedPrec.append(scorePrec2)
            scoreHit2=hit_upto_k(res2, valuedIDs, k)
            improvedHit.append(scoreHit2)
            print("")
            
#print(baseline)
print(mittel(baseline))
#print(improved)
print(mittel(improved))
print("Anzahl der verschiedenen Ereignisse: "+
      str(anzEreignisse))
print("Anzahl der verschiedenen Artikel: " + str(anzArtikel))
print(#'Baseline Precision score: '+
      str(mittel(baselinePrec)))
print(#'Improved Precision score: '+
      str(mittel(improvedPrec)))
print(#'Baseline Hit up to k score: '+
    str(mittel(baselineHit)))
print(#'Improved Hit up to k score: '+
    str(mittel(improvedHit)))





der maximale Score ist 7.0
baseline score ist 3.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 1.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 7.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 7.0
verbesserter score ist 0

der maximale Score ist 7.0
baseline score ist 3.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 3.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 7.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 3.0
verbesserter score ist 0.0

der maximale Score ist 7.0
baseline score ist 3.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 7.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 1.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 7.0
verbesserter score ist 7.0

der maximale Score ist 7.0
baseline score ist 7.0
verbesserter sco

In [43]:
#der Score wird ausgerechnet
article = data[topic]['data'][ereignisNummer]['articles'][artikelNummer]
ab= ArticleQueryBuilder(article)
queryBase=ab.baseline()
queryImprove= ab.simple_and_query() & ab.location_query() & ab.date_filter()
s = Search(using=client)
res1=s.query(queryBase).execute()
for value in res1:
    print(value['ID'])
res2=s.query(queryImprove).execute()
#for value in res2:
#    print(value['ID'])
valuedIDs=data[topic]['data'][ereignisNummer]['pr-ids']
k=3
maxScore=max_score(valuedIDs, k)
print("der maximale Score ist " +str(maxScore))
res1=first_k_results(res1,k)
res2=first_k_results(res2,k)
#res1[1]['ID']
score1=score(res1, valuedIDs,k)
print("baseline score ist " +str(score1))
score2=score(res2, valuedIDs,k)
print("verbesserter score ist " +str(score2))


3347872
1440995
3403353
977235
3308722
3071267
3866174
2794394
3062882
3553854
der maximale Score ist 14.9165082750002
baseline score ist 7.130929753571458
verbesserter score ist 14.9165082750002


In [145]:
int(res1[0]['ID'])==3617168

True

In [314]:
#Ausgabe der besten k Treffer der beiden Suchanfragen in der richtigen Reihenfolge
print("Baseline IDs")
for i in range(k):
    print(res1[i].meta.id)
        
print()
print("----------------------------------------------------")
print()

print("Improved IDs")
for i in range(k):
    print(res2[i].meta.id)

Baseline IDs
4s5TimUBlMjgtCulh19a
hc1SimUBlMjgtCulEHNw
bc5TimUBlMjgtCulUzXL

----------------------------------------------------

Improved IDs
4s5TimUBlMjgtCulh19a
Y85TimUBlMjgtCulh2xb
3c5TimUBlMjgtCulh19a


In [112]:
#die ersten k Treffer der beiden Suchanfragen werden in zufälliger Reihenfolge angezeigt
#Gibt eine bessere Version!!!
article = data[topic]['data'][ereignisNummer]['articles'][artikelNummer]
ab= ArticleQueryBuilder(article)
queryBase=ab.baseline()
queryImprove= ab.simple_and_query() & ab.location_query() & ab.date_filter()
s = Search(using=client)
res1=s.query(queryBase).execute()
res2=s.query(queryImprove).execute()
k=3

res1=first_k_results(res1,k)
res2=first_k_results(res2,k)
length=len(res1)+len(res2)
b=len(res1)
permutation=np.random.permutation(length)
#print(permutation)
for i in range(length):
    j=int(permutation[i])
    if j>=b:
        print(res2[j-b].ID)
        print(res2[(j-b)].title)
        print(res2[(j-b)].body)
        print()
        print("____________________________________________________")
        print("----------------------------------------------------")
        print()
    
    else:
        print(res1[(j)].ID)
        print(res1[(j)].title)
        print(res1[(j)].body)
        print()
        print("____________________________________________________")
        print("----------------------------------------------------")
        print()

<Response: [<Hit(police-prs/pr/205608): {'keywords': ['Polizei', 'Kriminalität'], 'officeName': 'Pol...}>, <Hit(police-prs/pr/605821): {'keywords': ['Polizei', 'Kriminalität'], 'officeID': '70246...}>, <Hit(police-prs/pr/295936): {'officeID': '70116', 'officeName': 'Bundespolizeidirektion ...}>, <Hit(police-prs/pr/1074540): {'keywords': ['Polizei', 'Kriminalität'], 'officeID': '70246...}>, <Hit(police-prs/pr/491088): {'keywords': ['Polizei', 'Kriminalität'], 'officeID': '70246...}>, <Hit(police-prs/pr/296607): {'ID': '3635426', 'keywords': ['Polizei', 'Kriminalität'], '...}>, <Hit(police-prs/pr/206760): {'keywords': ['Polizei', 'Kriminalität'], 'officeName': 'Pol...}>, <Hit(police-prs/pr/302718): {'ID': '3657815', 'keywords': ['Polizei', 'Kriminalität'], '...}>, <Hit(police-prs/pr/205108): {'keywords': ['Polizei', 'Kriminalität'], 'officeName': 'Pol...}>, <Hit(police-prs/pr/302649): {'ID': '3657622', 'keywords': ['Polizei', 'Kriminalität'], '...}>]>

3614839
POL-BI: Auseinandersetzungen am Gästeeingang der Schüco Arena
Bielefeld (ots) - MK / Bielefeld / Mitte - Vor dem Fußballspiel in
der Schüco Arena kam es am Montag, 17. April 2017, zu körperlichen 
Auseinandersetzungen im Bereich des Gästeeingangs. Daran beteiligt 
waren Stuttgarter Fans und der Ordnungsdienst. Polizeibeamte 
bereinigten die Situation unter Einsatz von Pfefferspray und des 
Einsatzmehrzweckstocks.

   Gegen 19:28 Uhr waren circa 300 Stuttgarter Fans aus ihren Bussen 
zum Gästeblock unterwegs. Ein Teil dieser Gruppe beabsichtigte 
offensichtlich, den Eingangsbereich zu überlaufen. Das versuchten 
Angehörige des Sicherheitsdienstes des Stadions zu unterbinden. Es 
entwickelte sich eine körperliche Auseinandersetzung zwischen einer 
größeren Fangruppe und dem eingesetzten Sicherheitsdienst. 
Hinzugerufene Beamte der Bereitschaftspolizei konnten die Situation 
bereinigen. Dazu setzten sie Pfefferspray und den 
Einsatzmehrzweckstock gegen gewalttätige Personen ein.



In [296]:
def construct_queries(article):
    builder = ArticleQueryBuilder(article)
    return {
        'and': builder.simple_and_query(),
        'or':builder.simple_or_query(),
        'and & date':builder.simple_and_query() & builder.date_filter(),
        'or & date':builder.simple_or_query() & builder.date_filter(),
        'and & loc':builder.simple_and_query() & builder.location_query(),
        'or & loc':builder.simple_or_query() & builder.location_query(),
        'and | loc':builder.simple_and_query() | builder.location_query(),
        'or | loc':builder.simple_or_query() | builder.location_query(),
        'and & loc & date':builder.simple_and_query() & builder.location_query() & builder.date_filter(),
        'or & loc & date':builder.simple_or_query() & builder.location_query() & builder.date_filter(),
        '(and | loc) & date':(builder.simple_and_query() | builder.location_query()) & builder.date_filter(),
        '(or | loc) & date':(builder.simple_or_query() | builder.location_query()) & builder.date_filter()
    }

In [297]:
def eval_query(query, pr_ids, client):
    res = Search(using=client).query(query).execute()
    print(res)
    return [(index, hit.meta.id) for index, hit in enumerate(res) if hit.meta.id in pr_ids]

In [298]:
def eval_event(event, client):
    results = defaultdict(dict)
    for article in event['articles']:
        queries = construct_queries(article)
        for name, query in queries.items(): 
            res = eval_query(query, event['pr-ids'], client)
            results[article['title']][name] = res
    return results

In [299]:
def eval_topic(topic, client):
    return [eval_event(event, client) for event in topic]

In [300]:
topic_results_2 = {topic_name: eval_topic(topic['data'], client) for 
                 topic_name, topic in data.items()}

<Response: [<Hit(police-prs/pr/U9tmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(police-prs/pr/DdtmimUBlMjgtCulU4SU): {'keywords': ['Polizei', 'Kriminalität'], 'body': '48147 Mün...}>, <Hit(police-prs/pr/uttmimUBlMjgtCulhays): {'body': 'Bochum (ots) -\n\n   Als Kerstin Wittmeier, Polize...}>, <Hit(police-prs/pr/ntJZimUBlMjgtCulzwZu): {'body': 'Hofheim (ots) - 1. Frau von Radler angefahren und ...}>, <Hit(police-prs/pr/o9tmimUBlMjgtCulhaes): {'keywords': ['Polizei'], 'body': 'Münster_BAB A 31_Gemeinde...}>, <Hit(police-prs/pr/fttmimUBlMjgtCulhaGr): {'body': 'Münster/Osnabrück/Hamm (ots) -\n\n   Erst vor zwei...}>, <Hit(police-prs/pr/ittmimUBlMjgtCulhaCr): {'body': 'Münster (ots) - Ein 22-jähriger Mann aus Albersloh...}>, <Hit(police-prs/pr/U9ZfimUBlMjgtCul8AbH): {'body': 'Münster (ots) - Ein 22-jähriger Mann aus Albersloh...}>, <Hit(police-prs/pr/5tJZimUBlMjgtCulzwdu): {'body': 'Münster (ots) - Ein 22-jähriger Mann aus Albersloh...}>, <Hit(po

<Response: [<Hit(police-prs/pr/f85UimUBlMjgtCulIZzK): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Freiburg ...}>, <Hit(police-prs/pr/zc5UimUBlMjgtCulIZvK): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Freiburg ...}>, <Hit(police-prs/pr/1s5UimUBlMjgtCulIZvK): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Freiburg ...}>, <Hit(police-prs/pr/U9tmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(police-prs/pr/F81SimUBlMjgtCulEH9y): {'body': 'Freiburg (ots) - Rickenbach. Zwischenzeitlich lieg...}>, <Hit(police-prs/pr/qs1SimUBlMjgtCulEH5y): {'body': 'Freiburg (ots) - Rickenbach - Derzeit laufen noch ...}>, <Hit(police-prs/pr/stJaimUBlMjgtCul-L_S): {'body': 'Berlin (ots) - Die terroristischen Anschläge in Be...}>, <Hit(police-prs/pr/_NNbimUBlMjgtCul8DZ0): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Berlin (o...}>, <Hit(police-prs/pr/Xd5qimUBlMjgtCulv9N1): {'body': 'Berlin (ots) - Die terroristischen Anschläge in Be...}>, <Hit(po

<Response: [<Hit(police-prs/pr/U9tmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(police-prs/pr/NthjimUBlMjgtCulA7BG): {'body': '48153 Münster (ots) - "Der ausgewogener Mix von \n...}>, <Hit(police-prs/pr/tNtmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(police-prs/pr/6tZgimUBlMjgtCuluKe4): {'keywords': ['Polizei'], 'body': 'Düsseldorf (ots) - Das er...}>, <Hit(police-prs/pr/otdhimUBlMjgtCulcx49): {'keywords': [], 'body': 'Heiligenhaus (ots) -\n\n   Am verg...}>, <Hit(police-prs/pr/ttplimUBlMjgtCulAHwD): {'keywords': ['Polizei'], 'body': 'Münster (ots) - Es ist sc...}>, <Hit(police-prs/pr/DdtmimUBlMjgtCulU4SU): {'keywords': ['Polizei', 'Kriminalität'], 'body': '48147 Mün...}>, <Hit(police-prs/pr/gNhjimUBlMjgtCulUNOX): {'body': '48147 Münster (ots) -\n\n\n   "Geschwindigkeit ist...}>, <Hit(police-prs/pr/mdxnimUBlMjgtCul0ciP): {'keywords': ['Polizei', 'Kriminalität'], 'body': '48147 Mün...}>, <Hit(po

<Response: [<Hit(police-prs/pr/U9tmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(police-prs/pr/DdtmimUBlMjgtCulU4SU): {'keywords': ['Polizei', 'Kriminalität'], 'body': '48147 Mün...}>, <Hit(police-prs/pr/tNtmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(police-prs/pr/P9tmimUBlMjgtCulhZ-r): {'body': 'Münster, Lengerich, Gronau (ots) - Am Donnerstagna...}>, <Hit(police-prs/pr/X9tmimUBlMjgtCulhZmr): {'body': 'Münster, Warendorf (ots) -\n\n   Am Donnerstagmorg...}>, <Hit(police-prs/pr/0dtmimUBlMjgtCulU3yT): {'keywords': ['Polizei'], 'body': 'Münster (ots) - Ein 52-jä...}>, <Hit(police-prs/pr/Q9tmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Recklingh...}>, <Hit(police-prs/pr/fttmimUBlMjgtCulhaGr): {'body': 'Münster/Osnabrück/Hamm (ots) -\n\n   Erst vor zwei...}>, <Hit(police-prs/pr/YNtmimUBlMjgtCulU3-T): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(po

<Response: [<Hit(police-prs/pr/U9tmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(police-prs/pr/DdtmimUBlMjgtCulU4SU): {'keywords': ['Polizei', 'Kriminalität'], 'body': '48147 Mün...}>, <Hit(police-prs/pr/tNtmimUBlMjgtCulU4KU): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Münster (...}>, <Hit(police-prs/pr/NtFYimUBlMjgtCul_Ktq): {'keywords': ['Polizei'], 'body': 'Münster (ots) - (Ergänzun...}>, <Hit(police-prs/pr/HNVfimUBlMjgtCulF1no): {'body': 'Münster (ots) - (Ergänzung zum Meldungstitel) Weni...}>, <Hit(police-prs/pr/ZNtmimUBlMjgtCulzMaS): {'keywords': ['Polizei'], 'body': 'Münster (ots) - (Ergänzun...}>, <Hit(police-prs/pr/uthjimUBlMjgtCulUMGW): {'body': '48147 Münster (ots) - "Viele Mordkommissionen und ...}>, <Hit(police-prs/pr/IdZgimUBlMjgtCul97qG): {'keywords': ['Polizei'], 'body': 'Münster (ots) - 9.735 Unf...}>, <Hit(police-prs/pr/V9VfimUBlMjgtCulF2Xr): {'body': 'Osnabrück (ots) - Nach den Vorfällen von Freitagna...}>, <Hit(po

...

<Response: [<Hit(police-prs/pr/ls1SimUBlMjgtCuly-wY): {'body': 'Ludwigsburg (ots) - Erdmannhausen: Verkehrsunfall ...}>, <Hit(police-prs/pr/WNplimUBlMjgtCulAIUE): {'keywords': ['Polizei'], 'body': 'Mettmann (ots) -\n\n   Am...}>, <Hit(police-prs/pr/oNReimUBlMjgtCulG3r3): {'body': 'Bergen, Kr. Celle (ots) - Bei einem Verkehrsunfall...}>, <Hit(police-prs/pr/2tZgimUBlMjgtCuluKS4): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Olsberg (...}>, <Hit(police-prs/pr/RNxnimUBlMjgtCul0dCQ): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Büren (ot...}>, <Hit(police-prs/pr/j9RdimUBlMjgtCulskYz): {'body': 'Wolfsburg (ots) - 16.07.14, 22.05 Uhr; Hattorf, La...}>, <Hit(police-prs/pr/Ht5qimUBlMjgtCulv9t2): {'body': 'Berka/Werra - Wartburg-Kreis (ots) - Zu einem Auff...}>, <Hit(police-prs/pr/G81SimUBlMjgtCulS5U1): {'body': 'Ludwigsburg (ots) - Korntal-Münchingen: Verkehrsun...}>, <Hit(police-prs/pr/e9dhimUBlMjgtCulcz9D): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Marsberg ...}>, <Hit(po

<Response: [<Hit(police-prs/pr/aNtmimUBlMjgtCulhZGq): {'body': 'Mönchengladbach (ots) - Gestern hat sich um 21.30 ...}>, <Hit(police-prs/pr/ettmimUBlMjgtCulhZOq): {'body': 'Mönchengladbach (ots) - Vergangenen Dienstag hat s...}>, <Hit(police-prs/pr/QtZfimUBlMjgtCul8BLI): {'body': 'Altkreis Norden (ots) - Verkehrsgeschehen\n\n   Gr...}>, <Hit(police-prs/pr/WdZfimUBlMjgtCul8A_H): {'body': 'Landkreis Verden (ots) -\n\n   Auto überschlägt si...}>, <Hit(police-prs/pr/edtmimUBlMjgtCulhZqr): {'body': 'Lippe (ots) - Auf der Straße Ostersiek in Alverdis...}>, <Hit(police-prs/pr/zdZfimUBlMjgtCul8A_I): {'body': 'Cuxhaven (ots) - Werkzeuge von Baustelle entwendet...}>, <Hit(police-prs/pr/cs9VimUBlMjgtCulVmdk): {'body': 'Konstanz (ots) - Konstanz\n\n   Gestürzte Radfahre...}>, <Hit(police-prs/pr/C9tmimUBlMjgtCulhaGr): {'body': 'Dortmund (ots) - Lfd. Nr.: 0694 Am Freitag, 11.5.2...}>, <Hit(police-prs/pr/fttmimUBlMjgtCulhYyp): {'body': 'Mönchengladbach (ots) - Am Montag (16.04.2018) hat...}>, <Hit(po

<Response: [<Hit(police-prs/pr/RNplimUBlMjgtCulVaxM): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Odenthal ...}>, <Hit(police-prs/pr/k9NbimUBlMjgtCul8DZ0): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'BAB 20 (o...}>, <Hit(police-prs/pr/JdplimUBlMjgtCulVaJK): {'body': 'Dortmund (ots) - Lfd. Nr.: 0817\n\n   Bei einem Ve...}>, <Hit(police-prs/pr/q91oimUBlMjgtCulyYOv): {'body': 'Insheim (ots) -\n\n   Am 23.07.2017, gegen 01:40 U...}>, <Hit(police-prs/pr/fs1SimUBlMjgtCulEGVt): {'body': 'Freiburg (ots) - Waldshut-Tiengen: Motorradfahrer ...}>, <Hit(police-prs/pr/F81SimUBlMjgtCulEHBv): {'body': 'Heilbronn (ots) - Schwarzach: Geschädigter nach Un...}>, <Hit(police-prs/pr/L95qimUBlMjgtCulv9V2): {'body': 'B 7/Creuzburg - Wartburg-Kreis (ots) - Gestern Abe...}>, <Hit(police-prs/pr/QdFZimUBlMjgtCulUtQw): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Friedberg...}>, <Hit(police-prs/pr/z81SimUBlMjgtCulEHlx): {'body': 'Heilbronn (ots) - Neckar-Odenwald-Kreis\n\n   Wall...}>, <Hit(po

<Response: [<Hit(police-prs/pr/A81SimUBlMjgtCulS44l): {'body': 'Neckarbischofsheim (ots) - Am Freitagabend, kurz v...}>, <Hit(police-prs/pr/ts9UimUBlMjgtCul1h13): {'body': 'Heilbronn (ots) - Buchen: Vier Wildunfälle innerha...}>, <Hit(police-prs/pr/Us5TimUBlMjgtCulUzTL): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Freiburg ...}>, <Hit(police-prs/pr/4tZhimUBlMjgtCulPOi2): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Düsseldor...}>, <Hit(police-prs/pr/n9lkimUBlMjgtCuln_a7): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Niederzie...}>, <Hit(police-prs/pr/q8xQimUBlMjgtCul_-YJ): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Walldorf/...}>, <Hit(police-prs/pr/RNplimUBlMjgtCulVaxM): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Odenthal ...}>, <Hit(police-prs/pr/mdNcimUBlMjgtCulQWZP): {'body': 'Bad Gandersheim (ots) - 37589 Kalefeld, K 602 und ...}>, <Hit(police-prs/pr/mc5TimUBlMjgtCulUynK): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Ulm (ots)...}>, <Hit(po

<Response: [<Hit(police-prs/pr/odtlimUBlMjgtCul4BZe): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Düsseldor...}>, <Hit(police-prs/pr/XNtmimUBlMjgtCulGDvJ): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Duisburg ...}>, <Hit(police-prs/pr/PttmimUBlMjgtCulGDrJ): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Duisburg/...}>, <Hit(police-prs/pr/VdtmimUBlMjgtCulGDrJ): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Essen (ot...}>, <Hit(police-prs/pr/CttlimUBlMjgtCul4BRe): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Düsseldor...}>, <Hit(police-prs/pr/L9tmimUBlMjgtCulGDnJ): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Köln (ots...}>, <Hit(police-prs/pr/rNtmimUBlMjgtCulGCfG): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Duisburg ...}>, <Hit(police-prs/pr/ctFZimUBlMjgtCulz_Vt): {'keywords': ['Polizei', 'Kriminalität'], 'body': 'Kassel (o...}>, <Hit(police-prs/pr/hdVfimUBlMjgtCulx87m): {'body': 'Rotenburg (ots) -\n\n   Stau auf der Hansalinie na...}>, <Hit(po

In [301]:
topic_results_2.keys()

dict_keys(['straftaten-anschlaege', 'unfaelle', 'immigration', 'grossveranstaltungen', 'demonstrationen', 'sport-fussball'])

In [302]:
def relevant_upto_k(query_result, k):
    relevant = 0
    for hit in query_result:
        rank, _ = hit
        if rank + 1 <= k:
            relevant = 1
    return relevant

In [303]:
def calc_mean_relevant_upto_k_by_topic(topic_result, k):
    query_result = []
    for event_result in topic_result:
        for title, query_dict in event_result.items():
            for query_type, results in query_dict.items():
                query_results.append(results)
                
    return sum(relevant_upto_k(result) for result in query_results) / len(query_results)
        
    

In [304]:
def precision_at_k(query_result, k):
    precision = 0
    for hit in query_result:
        rank, _ = hit
        if rank + 1 <= k:
            precision += 1
    return precision / k # We assume that always 
                         # at least k documents have been retrieved

In [305]:
def calc_mean_precision_at_k_by_topic(topic_result, k):
    query_results = []
    for event_result in topic_result:
        for title, query_dict in event_result.items():
            for query_type, results in query_dict.items():
                query_results.append(results)
    
    return sum(precision_at_k(result) for result in query_results) \
        / len(query_results)

In [306]:
precision_at_1_by_topic = {
    topic: calc_precision_at_k_by_topic(topic_results_2[topic], 1)
    for topic in data_file_names
}

NameError: name 'calc_precision_at_k_by_topic' is not defined

In [307]:
precision_at_1_by_topic

NameError: name 'precision_at_1_by_topic' is not defined

In [308]:
def split_topic_results_into_query_types(topic_results):
    query_type_results = defaultdict(list)
    for topic_name, topic_result in topic_results.items():
        for event_result in topic_result:
            for title, query_dict in event_result.items():
                for query_type, results in query_dict.items():
                    query_type_results[query_type].append(results)
    return query_type_results

In [309]:
len([el for el in split_topic_results_into_query_types(topic_results_2)['or'] if len(el) > 0 and el[0][0]==0])

0

In [434]:
len([el for el in split_topic_results_into_query_types(topic_results_2)['or'] if len(el) > 0 and el[0][0]==0])

35

In [310]:
split_topic_results_into_query_types(topic_results_2)['or']

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [441]:
def calc_precision_at_k_by_query_type(topic_results, k):
    query_type_results = split_topic_results_into_query_types(topic_results)
    precisions = {}
    for query_type, results_by_type in query_type_results.items():
        sum_precision_at_k = 0
        for results in results_by_type:
            sum_precision_at_k += precision_at_k(results, k)
        precisions[query_type] = sum_precision_at_k / len(results_by_type)
    return precisions

In [451]:
calc_precision_at_k_by_query_type(topic_results_2, 3)

{'and': 0.2753623188405797,
 'or': 0.31884057971014496,
 'and & date': 0.2971014492753623,
 'or & date': 0.33333333333333337,
 'and & loc': 0.3043478260869565,
 'or & loc': 0.34782608695652173,
 'and | loc': 0.29710144927536225,
 'or | loc': 0.34057971014492755,
 'and & loc & date': 0.32608695652173914,
 'or & loc & date': 0.3695652173913043,
 '(and | loc) & date': 0.31884057971014496,
 '(or | loc) & date': 0.36231884057971014}

In [ ]:
def calc_mean_mrr_by_query(topic_results):
    pass

In [362]:
topic_results

{'grossveranstaltungen': [defaultdict(dict,
              {'Rhein in Flammen: Polizei ermittelt wegen sexueller Übergriffe': {'and': [(0,
                  'NfDKVmUB_NHj735hjcZN')],
                'or': [(0, 'NfDKVmUB_NHj735hjcZN')],
                'and & date': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (2, 'jPDKVmUB_NHj735hjb5L')],
                'or & date': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (3, 'jPDKVmUB_NHj735hjb5L')],
                'and & loc': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (7, 'jPDKVmUB_NHj735hjb5L')],
                'or & loc': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (8, 'jPDKVmUB_NHj735hjb5L')],
                'and | loc': [(0, 'NfDKVmUB_NHj735hjcZN')],
                'or | loc': [(0, 'NfDKVmUB_NHj735hjcZN')],
                'and & loc & date': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (1, 'jPDKVmUB_NHj735hjb5L')],
                'or & loc & date': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (1, 'jPDKVmUB_NHj735hjb5L')

In [464]:
art = Article('http://www.wn.de/Muenster/3246264-Fahrzeug-faehrt-in-Menschenmenge-Tote-und-Verletzte-nach-Anschlag'
             ,language='de')

In [471]:
art.download()
art.parse()
art.nlp()

In [472]:
art.title

'Fahrzeug fährt in Menschenmenge Tote und Verletzte nach Anschlag'

In [475]:
art.summary

'Tote und Verletzte nach Anschlag am Kiepenkerl 1/53 Kleinlaster fährt in Menschenmenge Foto: privatKleinlaster fährt in Menschenmenge Foto: Oliver WernerMenschen stehen vor den Gaststätten Kiepenkerl, kurz nachdem ein Fahrzeug in das Straßencafe gefahren war.\nFoto: Ina Fassbender/dpaFoto: David Young/dpaKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerFoto: Bernd Thissen/dpaKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerAm Abend nach der Bluttat in Münster versammelten sich Menschen am Aasee und entzündeten Kerzen.\nFoto: Friso Gentsch/dpaKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerFoto: Bernd Thissen/dpaKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: